In [1]:
#Importing packages
#
# You can delete whatever package(s) you chose..
#

import math as m
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Leave This code here and lookup at pandas documentation 
# if you need to know about chained assignments
pd.options.mode.chained_assignment = None  # default='warn'

# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 99

In [4]:
gm23 = pd.read_excel('MBB_GMS_DATA_HoopsIQ.xlsx', sheet_name='PlayerStats_2_06_20')
gm23

,#,Last,First,Min,Pos,Pts,+/-,FGM,FGA,FG%,2FM,2FA,2F%,3FM,3FA,3F%,FTM,FTA,FT%,ORb,DRb,Reb,Ast,TO,Stl,Blk,PF,Pace,ORTG,DRTG,NRTG,TPos,TPE,TPo%,eFG%,TS%,2FR,3FR,FTR,ORB%,DRB%,REB%,ORbD,AST%,ATR,TO%,APts,CPts,CP%
0,1,BAXTER,HUGH,8.916667,14,2,1,1,3,33.333333,1,3,33.333333,0,0,-1.000000,0,0,-1.000000,1,2,3,0,0,0,0,0,62.803738,114.285714,100.000000,14.285714,3,66.666667,20.000000,33.333333,33.333333,100.000000,0.000000,0.000000,11.111111,18.181818,15.000000,-3,0.000000,-1.0,0.000000,0,2,12.500000
1,11,EVERMAN,SCOTT,31.083333,54,12,7,5,8,62.500000,3,5,60.000000,2,3,66.666667,0,0,-1.000000,2,1,3,1,2,1,1,3,69.490617,122.222222,107.272727,14.949495,10,120.000000,16.393443,75.000000,75.000000,62.500000,37.500000,0.000000,7.142857,4.347826,5.882353,3,5.882353,0.5,20.000000,3,15,22.727273
2,3,HADLEY,GABE,27.316667,45,9,-9,3,7,42.857143,2,3,66.666667,1,4,25.000000,2,3,66.666667,0,2,2,2,0,0,0,2,65.893838,102.222222,119.565217,-17.342995,8,112.500000,16.000000,50.000000,56.250000,42.857143,57.142857,42.857143,0.000000,9.090909,3.921569,-1,15.384615,-1.0,0.000000,5,14,30.434783
3,32,HOWELL,MIKEY,38.166667,65,0,8,0,4,0.000000,0,3,0.000000,0,1,0.000000,0,0,-1.000000,0,4,4,9,2,1,0,3,68.122271,123.076923,109.090909,13.986014,6,0.000000,8.219178,0.000000,0.000000,75.000000,25.000000,0.000000,0.000000,12.903226,6.060606,1,33.333333,4.5,33.333333,25,25,31.250000
4,14,OSHITA,CHRISTIAN,28.416667,48,20,18,7,16,43.750000,2,7,28.571429,5,9,55.555556,1,2,50.000000,1,5,6,1,0,1,0,4,67.565982,139.583333,102.083333,37.500000,17,117.647059,31.481482,59.375000,58.823529,43.750000,56.250000,12.500000,4.166667,17.857143,11.538462,-1,6.666667,-1.0,0.000000,3,23,34.328358
5,10,PETERSON,CONNOR,3.200000,6,0,1,0,0,-1.000000,0,0,-1.000000,0,0,-1.000000,0,0,-1.000000,0,1,1,0,1,0,0,1,75.000000,83.333333,66.666667,16.666667,1,0.000000,16.666667,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,25.000000,16.666667,-1,0.000000,0.0,100.000000,0,0,0.000000
6,2,ROBERTS,TYRELL,38.633333,65,30,7,9,17,52.941176,2,7,28.571429,7,10,70.000000,5,9,55.555556,1,5,6,2,1,2,0,3,67.299396,121.538462,105.882353,15.656109,22,136.363636,30.136986,73.529412,71.428571,41.176471,58.823529,52.941176,2.702703,15.151515,8.571429,0,11.111111,2.0,4.545455,6,36,45.569620
7,5,SULLIVAN,MAREK,24.266667,43,9,7,3,6,50.000000,2,3,66.666667,1,3,33.333333,2,2,100.000000,0,4,4,2,2,5,0,4,70.879121,118.604651,95.652174,22.952477,9,100.000000,18.750000,58.333333,64.285714,50.000000,50.000000,33.333333,0.000000,22.222222,10.256410,2,13.333333,1.0,22.222222,6,15,29.411765


In [18]:
# gm23['gmsc'] = gm23['Pts'] + 0.4 * gm23['FGM'] - 0.7 * gm23['FGA'] - 0.4*(gm23['FTA'] - gm23['FTM']) + 0.7 * gm23['ORb'] + 0.3 * gm23['DRb'] + gm23['Stl'] + 0.7 * gm23['Ast'] + 0.7 * gm23['Blk'] - 0.4 * gm23['PF'] - gm23['TO']

In [19]:
gm23

,#,Last,First,Min,Pos,Pts,+/-,FGM,FGA,FG%,2FM,2FA,2F%,3FM,3FA,3F%,FTM,FTA,FT%,ORb,DRb,Reb,Ast,TO,Stl,Blk,PF,Pace,ORTG,DRTG,NRTG,TPos,TPE,TPo%,eFG%,TS%,2FR,3FR,FTR,ORB%,DRB%,REB%,ORbD,AST%,ATR,TO%,APts,CPts,CP%,gmsc
0,1,BAXTER,HUGH,8.916667,14,2,1,1,3,33.333333,1,3,33.333333,0,0,-1.000000,0,0,-1.000000,1,2,3,0,0,0,0,0,62.803738,114.285714,100.000000,14.285714,3,66.666667,20.000000,33.333333,33.333333,100.000000,0.000000,0.000000,11.111111,18.181818,15.000000,-3,0.000000,-1.0,0.000000,0,2,12.500000,1.6
1,11,EVERMAN,SCOTT,31.083333,54,12,7,5,8,62.500000,3,5,60.000000,2,3,66.666667,0,0,-1.000000,2,1,3,1,2,1,1,3,69.490617,122.222222,107.272727,14.949495,10,120.000000,16.393443,75.000000,75.000000,62.500000,37.500000,0.000000,7.142857,4.347826,5.882353,3,5.882353,0.5,20.000000,3,15,22.727273,9.3
2,3,HADLEY,GABE,27.316667,45,9,-9,3,7,42.857143,2,3,66.666667,1,4,25.000000,2,3,66.666667,0,2,2,2,0,0,0,2,65.893838,102.222222,119.565217,-17.342995,8,112.500000,16.000000,50.000000,56.250000,42.857143,57.142857,42.857143,0.000000,9.090909,3.921569,-1,15.384615,-1.0,0.000000,5,14,30.434783,6.1
3,32,HOWELL,MIKEY,38.166667,65,0,8,0,4,0.000000,0,3,0.000000,0,1,0.000000,0,0,-1.000000,0,4,4,9,2,1,0,3,68.122271,123.076923,109.090909,13.986014,6,0.000000,8.219178,0.000000,0.000000,75.000000,25.000000,0.000000,0.000000,12.903226,6.060606,1,33.333333,4.5,33.333333,25,25,31.250000,2.5
4,14,OSHITA,CHRISTIAN,28.416667,48,20,18,7,16,43.750000,2,7,28.571429,5,9,55.555556,1,2,50.000000,1,5,6,1,0,1,0,4,67.565982,139.583333,102.083333,37.500000,17,117.647059,31.481482,59.375000,58.823529,43.750000,56.250000,12.500000,4.166667,17.857143,11.538462,-1,6.666667,-1.0,0.000000,3,23,34.328358,13.5
5,10,PETERSON,CONNOR,3.200000,6,0,1,0,0,-1.000000,0,0,-1.000000,0,0,-1.000000,0,0,-1.000000,0,1,1,0,1,0,0,1,75.000000,83.333333,66.666667,16.666667,1,0.000000,16.666667,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,25.000000,16.666667,-1,0.000000,0.0,100.000000,0,0,0.000000,-1.1
6,2,ROBERTS,TYRELL,38.633333,65,30,7,9,17,52.941176,2,7,28.571429,7,10,70.000000,5,9,55.555556,1,5,6,2,1,2,0,3,67.299396,121.538462,105.882353,15.656109,22,136.363636,30.136986,73.529412,71.428571,41.176471,58.823529,52.941176,2.702703,15.151515,8.571429,0,11.111111,2.0,4.545455,6,36,45.569620,23.5
7,5,SULLIVAN,MAREK,24.266667,43,9,7,3,6,50.000000,2,3,66.666667,1,3,33.333333,2,2,100.000000,0,4,4,2,2,5,0,4,70.879121,118.604651,95.652174,22.952477,9,100.000000,18.750000,58.333333,64.285714,50.000000,50.000000,33.333333,0.000000,22.222222,10.256410,2,13.333333,1.0,22.222222,6,15,29.411765,10.0
